<a href="https://colab.research.google.com/github/Konstantin-Larionov/ds_sb_2020_homework/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Part_1_Intro_to_SQL_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем модуль для работы с базой SQLite

In [ ]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [ ]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [ ]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
os.path.exists(path)

True

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [ ]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [ ]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [ ]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [ ]:
conn.commit()
conn.close()

In [ ]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [ ]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

ProgrammingError: ignored

Почему на выходе получили только 1 комманду?

In [ ]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

ProgrammingError: ignored

In [ ]:
cursor.close()
conn.close()

ProgrammingError: ignored

In [ ]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [ ]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [ ]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [ ]:
rows

[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [ ]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50


In [ ]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [ ]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [1]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-15 18:30:40,927 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-15 18:30:40,929 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:30:40,932 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-15 18:30:40,933 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [2]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [3]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4543 sha256=4388f6b2826ccae3ec2e51a15ff0f194028df1bfaffab8d6b1ba53299e46cd85
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [4]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
#Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [5]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [6]:
Purchase.ItemId.name

'ItemId'

In [7]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [8]:
Base.metadata.create_all(engine)

2020-12-15 18:31:36,453 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-15 18:31:36,454 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:31:36,457 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-15 18:31:36,458 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:31:36,460 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-15 18:31:36,463 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:31:36,465 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-15 18:31:36,467 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:31:36,468 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-15 18:31:36,469 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:31:36,470 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-15 18:31:36,471 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:31:36,475 INFO sqlalchemy.engine.base.Engine 
CREATE

In [9]:
engine.table_names()

2020-12-15 18:31:40,296 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-15 18:31:40,297 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [10]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-15 18:31:43,318 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 18:31:43,320 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 18:31:43,321 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-15 18:31:43,326 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [11]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-15 18:31:53,002 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-15 18:31:53,004 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [12]:
display_results(customers_query)

2020-12-15 18:32:00,031 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-15 18:32:00,033 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [13]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-15 18:32:05,669 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 18:32:05,671 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 18:32:05,673 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-15 18:32:05,676 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-15 18:32:10,072 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 18:32:10,073 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 18:32:10,075 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-15 18:32:10,076 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-15 18:32:14,965 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 18:32:14,971 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-15 18:32:14,973 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-15 18:32:14,975 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [16]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-15 18:32:18,012 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-15 18:32:18,013 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com
3,Lisa,lisa@White.com


In [17]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c61308>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c61360>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c610f8>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c61258>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c61410>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c613b8>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f2db4c61150>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7f2db4c689b0; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=Engine(sqlite:///:memory:)), Column('CustomerId', Integer(), table=<customers>, primary_key=True

In [18]:
list_info = session.query(Customer).all()

2020-12-15 18:32:32,644 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 18:32:32,650 INFO sqlalchemy.engine.base.Engine SELECT customers."CustomerId" AS "customers_CustomerId", customers."FirstName" AS "customers_FirstName", customers."LastName" AS "customers_LastName", customers."Company" AS "customers_Company", customers."Address" AS "customers_Address", customers."Phone" AS "customers_Phone", customers."Email" AS "customers_Email" 
FROM customers
2020-12-15 18:32:32,653 INFO sqlalchemy.engine.base.Engine ()


In [19]:
list_info

In [20]:
for id_ in list_info:
    print(id_.CustomerId)

1
2
3
4


In [21]:
first_info = session.query(Customer.CustomerId)

In [22]:
first_info

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [23]:
from sqlalchemy import *

In [24]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email]).order_by(asc(Customer.FirstName), desc(Customer.LastName))
#customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-15 18:33:02,204 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC
2020-12-15 18:33:02,206 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [25]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-15 18:33:05,683 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC, customers."FirstName" DESC
2020-12-15 18:33:05,685 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [26]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-15 18:33:08,930 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-15 18:33:08,931 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [27]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(and_(
                    users_table.c.name == 'wendy',
                    Customer.FirstName == ).order_by(asc(Customer.LastName))

In [28]:
display_results(customers_query)

2020-12-15 18:33:16,138 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-15 18:33:16,140 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [29]:
import pandas as pd

In [30]:
#выведем название нашей таблицы
Item.__tablename__

'items'

In [31]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-15 18:33:33,540 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-15 18:33:33,542 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:33:33,543 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-15 18:33:33,545 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:33:33,546 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-15 18:33:33,548 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [32]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
sql = """
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
"""
df = pd.read_sql(sql, conn)

df.head()

2020-12-15 18:33:36,494 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-15 18:33:36,497 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:33:36,499 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-15 18:33:36,500 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:33:36,502 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-15 18:33:36,503 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [33]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-15 18:33:40,769 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-15 18:33:40,771 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:33:40,772 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-15 18:33:40,774 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 18:33:40,775 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-15 18:33:40,776 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.


<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

!!!!!Решенные задачи по SQL на сайте sql-ex в виде принстскрина находятся в репозитории в файле с названием безымянный


In [45]:
sponge = Item(
    Name='Sponge', 
    Price=100)
session.add(sponge)
session.commit()

2020-12-15 19:21:26,501 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:21:26,503 INFO sqlalchemy.engine.base.Engine ('Sponge', 100.0)
2020-12-15 19:21:26,505 INFO sqlalchemy.engine.base.Engine COMMIT


In [46]:
Bob = Item(
    Name='Bob', 
    Price=110)
session.add(Bob)
session.commit()

2020-12-15 19:22:08,864 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:22:08,865 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:22:08,866 INFO sqlalchemy.engine.base.Engine ('Bob', 110.0)
2020-12-15 19:22:08,869 INFO sqlalchemy.engine.base.Engine COMMIT


In [47]:
Square = Item(
    Name='Square', 
    Price=120)
session.add(Square)
session.commit()

2020-12-15 19:22:55,452 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:22:55,454 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:22:55,458 INFO sqlalchemy.engine.base.Engine ('Square', 120.0)
2020-12-15 19:22:55,465 INFO sqlalchemy.engine.base.Engine COMMIT


In [48]:
Pants = Item(
    Name='Pants', 
    Price=115)
session.add(Pants)
session.commit()

2020-12-15 19:23:32,520 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:23:32,521 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:23:32,523 INFO sqlalchemy.engine.base.Engine ('Pants', 115.0)
2020-12-15 19:23:32,528 INFO sqlalchemy.engine.base.Engine COMMIT


In [49]:
ahahaha = Item(
    Name='ahahaha', 
    Price=5000)
session.add(ahahaha)
session.commit()

2020-12-15 19:24:46,056 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:24:46,060 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-15 19:24:46,061 INFO sqlalchemy.engine.base.Engine ('ahahaha', 5000.0)
2020-12-15 19:24:46,064 INFO sqlalchemy.engine.base.Engine COMMIT


In [50]:
sqlitems = """
SELECT *
FROM
    Items

;
"""
df = pd.read_sql(sqlitems, conn)
df

2020-12-15 19:26:02,266 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT *
FROM
    Items

;
")
2020-12-15 19:26:02,267 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:26:02,268 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT *
FROM
    Items

;
")
2020-12-15 19:26:02,270 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:26:02,271 INFO sqlalchemy.engine.base.Engine 
SELECT *
FROM
    Items

;

2020-12-15 19:26:02,272 INFO sqlalchemy.engine.base.Engine ()


,ItemId,Name,Price
0,1,Sponge,100
1,2,Bob,110
2,3,Square,120
3,4,Pants,115
4,5,ahahaha,5000


In [71]:
import datetime as dt

Purchase1 = Purchase(
    ItemId=1,
    CustomerId=1,
    Date=dt.datetime(2020,5,1))
session.add(Purchase1)
session.commit()

2020-12-15 19:49:49,969 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:49:49,971 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:49:49,971 INFO sqlalchemy.engine.base.Engine (1, 1, '2020-05-01 00:00:00.000000')
2020-12-15 19:49:49,973 INFO sqlalchemy.engine.base.Engine COMMIT


In [72]:
Purchase2 = Purchase(
    ItemId=2,
    CustomerId=2,
    Date=dt.datetime(2020,5,2))
session.add(Purchase2)
session.commit()

2020-12-15 19:50:19,536 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:50:19,539 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:50:19,540 INFO sqlalchemy.engine.base.Engine (2, 2, '2020-05-02 00:00:00.000000')
2020-12-15 19:50:19,543 INFO sqlalchemy.engine.base.Engine COMMIT


In [73]:
Purchase3 = Purchase(
    ItemId=3,
    CustomerId=3,
    Date=dt.datetime(2020,5,3))
session.add(Purchase3)
session.commit()

2020-12-15 19:50:45,494 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:50:45,496 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:50:45,496 INFO sqlalchemy.engine.base.Engine (3, 3, '2020-05-03 00:00:00.000000')
2020-12-15 19:50:45,498 INFO sqlalchemy.engine.base.Engine COMMIT


In [74]:
Purchase4 = Purchase(
    ItemId=4,
    CustomerId=4,
    Date=dt.datetime(2020,5,4))
session.add(Purchase4)
session.commit()

2020-12-15 19:51:23,660 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:51:23,661 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:51:23,663 INFO sqlalchemy.engine.base.Engine (4, 4, '2020-05-04 00:00:00.000000')
2020-12-15 19:51:23,666 INFO sqlalchemy.engine.base.Engine COMMIT


In [75]:
Purchase5 = Purchase(
    ItemId=5,
    CustomerId=4,
    Date=dt.datetime(2020,5,5))
session.add(Purchase5)
session.commit()

2020-12-15 19:51:44,819 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-15 19:51:44,823 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-15 19:51:44,825 INFO sqlalchemy.engine.base.Engine (5, 4, '2020-05-05 00:00:00.000000')
2020-12-15 19:51:44,829 INFO sqlalchemy.engine.base.Engine COMMIT


In [76]:
LisaID=select([Customer.CustomerId])
LisaID=LisaID.where(Customer.FirstName == 'Lisa')

display_results(LisaID)

2020-12-15 19:51:50,159 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."CustomerId" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-15 19:51:50,161 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,CustomerId
0,2
1,4


In [84]:
Lisaid_query=select([Purchase.PurchaseId,Purchase.ItemId,Purchase.CustomerId,Purchase.Date])
Lisaid_query=Lisaid_query.where((Purchase.CustomerId == LisaID)|(Purchase.CustomerId == 4))
display_results(Lisaid_query)

2020-12-15 20:04:37,821 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."PurchaseId", purchases."ItemId", purchases."CustomerId", purchases."Date" 
FROM purchases 
WHERE purchases."CustomerId" = (SELECT customers."CustomerId" 
FROM customers 
WHERE customers."FirstName" = ?) OR purchases."CustomerId" = ?
2020-12-15 20:04:37,823 INFO sqlalchemy.engine.base.OptionEngine ('Lisa', 4)


,PurchaseId,ItemId,CustomerId,Date
0,3,2,2,2020-05-02
1,5,4,4,2020-05-04
2,6,5,4,2020-05-05


In [78]:
sql = """
SELECT *
FROM
    Purchases
WHERE CustomerId in (SELECT CustomerId FROM customers WHERE FirstName = "Lisa")
;
"""
df = pd.read_sql(sql, conn)
df

2020-12-15 19:52:20,416 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT *
FROM
    Purchases
WHERE CustomerId in (SELECT CustomerId FROM customers WHERE FirstName = ""Lisa"")
;
")
2020-12-15 19:52:20,417 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:52:20,418 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT *
FROM
    Purchases
WHERE CustomerId in (SELECT CustomerId FROM customers WHERE FirstName = ""Lisa"")
;
")
2020-12-15 19:52:20,419 INFO sqlalchemy.engine.base.Engine ()
2020-12-15 19:52:20,420 INFO sqlalchemy.engine.base.Engine 
SELECT *
FROM
    Purchases
WHERE CustomerId in (SELECT CustomerId FROM customers WHERE FirstName = "Lisa")
;

2020-12-15 19:52:20,421 INFO sqlalchemy.engine.base.Engine ()


,PurchaseId,ItemId,CustomerId,Date
0,3,2,2,2020-05-02 00:00:00.000000
1,5,4,4,2020-05-04 00:00:00.000000
2,6,5,4,2020-05-05 00:00:00.000000
